In [193]:
import os
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split

In [196]:
destination =  os.environ.get('LR_Destination')
ChurnData = pd.read_csv(str(destination + '/' + 'WA_Fn-UseC_-Telco-Customer-Churn.csv'))

TypeError: 'tuple' object is not callable

#### Although Decision Trees can train on complex non-linear relationships to determine the relationship between a feature and a  target variable, it can't understand categorical features that are expressed through strings like 'Yes' or 'No' / 'Female' or 'Male' so we must map to these to numerical values without necessarily needing to create indicator variables unlike with building a logistic regression model

In [ ]:
#First drop off customerID column
if 'customerID' in ChurnData.columns.values:
     ChurnData = ChurnData.drop(['customerID'],axis=1)
else:
     1==1

#Converting Total Charges to Float
ChurnData['TotalCharges'] = pd.to_numeric(ChurnData['TotalCharges'], errors='coerce').fillna(pd.to_numeric(ChurnData['TotalCharges'], errors='coerce').mean())


unique_values_dict = {}
for cols in ChurnData.columns.values:
    if ChurnData[cols].dtypes != 'int64' and ChurnData[cols].dtypes != 'float64':
          unique_values_dict[cols] = ChurnData[cols].unique()

In [183]:
for i in unique_values_dict:
    remapped_values = {}
    names_and_remapped_values = {}
    #  print(i, unique_values_dict[i])

    #for i in range of length of values in key:values for gender
    for iterator in range(len(unique_values_dict[i])):
        remapped_values[unique_values_dict[i][iterator]] = iterator

        names_and_remapped_values[i] = remapped_values 

    for key in names_and_remapped_values:
        col_name = '\'' + key + '\''
        print(col_name)

'gender'
'Partner'
'Dependents'
'PhoneService'
'MultipleLines'
'InternetService'
'OnlineSecurity'
'OnlineBackup'
'DeviceProtection'
'TechSupport'
'StreamingTV'
'StreamingMovies'
'Contract'
'PaperlessBilling'
'PaymentMethod'
'Churn'


In [188]:
ChurnData

0       0
1       1
2       1
3       1
4       0
       ..
7038    1
7039    0
7040    0
7041    1
7042    1
Name: gender, Length: 7043, dtype: int64